build a basic chatbot (graph api functionality)

In [4]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class state(TypedDict):
    messages : Annotated[list,add_messages]
graphbuilder = StateGraph(state)

TypeError: 'type' object is not subscriptable